In [23]:
import numpy as np
import pandas as pd
import sklearn

在实际项目中，我们经常需要将数据拆分为训练集、测试集、验证集等。本部分介绍了如何拆分数据集，主要包括自定义实现以及sklearn实现2种方式。

## 1、数据乱序 shuffle() & permutation()
拆分前，一般会先对数据进行随机排序。

numpy.random中有shuffle()和permutation()2个函数均可用于对数据进行乱序。主要区别在于：
* shuffle()直接对原数据进行重排，无返回值。
* permutation()复制原数据，然后再重排，返回重排后的数组。原数据没有任何变化。

生成数据：

In [24]:
data = np.arange(100).reshape(10,-1)
print(data)

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


使用permutation()重排：

In [25]:
x = np.random.permutation(data)
print(x)
print(data)

[[ 0  1  2  3  4  5  6  7  8  9]
 [40 41 42 43 44 45 46 47 48 49]
 [60 61 62 63 64 65 66 67 68 69]
 [90 91 92 93 94 95 96 97 98 99]
 [50 51 52 53 54 55 56 57 58 59]
 [70 71 72 73 74 75 76 77 78 79]
 [10 11 12 13 14 15 16 17 18 19]
 [30 31 32 33 34 35 36 37 38 39]
 [20 21 22 23 24 25 26 27 28 29]
 [80 81 82 83 84 85 86 87 88 89]]
[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]


使用shuffle()重排：

In [26]:
np.random.shuffle(data)
print(data)

[[20 21 22 23 24 25 26 27 28 29]
 [50 51 52 53 54 55 56 57 58 59]
 [40 41 42 43 44 45 46 47 48 49]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [30 31 32 33 34 35 36 37 38 39]
 [10 11 12 13 14 15 16 17 18 19]
 [90 91 92 93 94 95 96 97 98 99]
 [60 61 62 63 64 65 66 67 68 69]
 [ 0  1  2  3  4  5  6  7  8  9]]


## 2、自定义拆分数据

### 2.1 基本方式
我们使用自定义函数的方式，随机抽取20%的样本作为测试集：

In [45]:

#如果是pandas数据
def split_train_test(data, test_ratio):
    shuffle_indices = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)                                                     
    training_idx, test_idx = shuffle_indices[test_size:], shuffle_indices[:test_size]
    return data.iloc[training_idx], data.iloc[test_idx]

trining_data, test_data = split_train_test(pd.DataFrame(x), 0.2)
print(test_data)


    0   1   2   3   4   5   6   7   8   9
0   0   1   2   3   4   5   6   7   8   9
1  40  41  42  43  44  45  46  47  48  49


In [40]:
#如果是numpy数据,建议使用pd.DataFrame()先转换为pandas数据，也可以使用以下方式：
def split_train_test_np(data, test_ratio):
    shuffle_indeices = np.random.permutation(data.shape[0])
    test_size = int(data.shape[0] * test_ratio)                                                     
    training_idx, test_idx = shuffle_indeices[test_size:], shuffle_indeices[:test_size]
    return data[training_idx], data[test_idx]

trining_data, test_data = split_train_test_np(x, 0.2)
print(test_data)

[[70 71 72 73 74 75 76 77 78 79]
 [90 91 92 93 94 95 96 97 98 99]]


### 2.2 固定样本
运行上面的代码会发现，每次运行时得到的样本都不同，我们可以增加一个随机种子，使得每次随机结果都相同。

In [48]:
np.random.seed(42)

trining_data, test_data = split_train_test(pd.DataFrame(x), 0.2)
print(test_data)

    0   1   2   3   4   5   6   7   8   9
8  20  21  22  23  24  25  26  27  28  29
1  40  41  42  43  44  45  46  47  48  49


### 2.3 样本集更新导致的测试集变化
上述虽然解决了每次运行得到不同随机结果的问题，但如果由于样本增加或者减少时，一个样本有可能会被重新划分到另一个数据集。

解决这个问题的思路是：计算每个实例标识符的hash值，如果hash小于最大值的20%，则将实例放入测试集：

In [50]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [60]:
# 由于housing数据没有index，所以我们使用行索引作为ID：
x_pd = pd.DataFrame(x)
x_with_id = x_pd.reset_index()
split_train_test_by_id(x_with_id, 0.2, 'index')

(   index   0   1   2   3   4   5   6   7   8   9
 0      0   0   1   2   3   4   5   6   7   8   9
 1      1  40  41  42  43  44  45  46  47  48  49
 3      3  90  91  92  93  94  95  96  97  98  99
 4      4  50  51  52  53  54  55  56  57  58  59
 6      6  10  11  12  13  14  15  16  17  18  19
 7      7  30  31  32  33  34  35  36  37  38  39
 8      8  20  21  22  23  24  25  26  27  28  29
 9      9  80  81  82  83  84  85  86  87  88  89,
    index   0   1   2   3   4   5   6   7   8   9
 2      2  60  61  62  63  64  65  66  67  68  69
 5      5  70  71  72  73  74  75  76  77  78  79)

## 3、使用sklearn的方式

其实sklearn也提供了一个函数用于同样的功能：

In [63]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(x_pd, test_size=0.2, random_state=42)
test_set.head()

,0,1,2,3,4,5,6,7,8,9
8,20,21,22,23,24,25,26,27,28,29
1,40,41,42,43,44,45,46,47,48,49
